In [71]:
import numpy as np
import pandas as pd

#### Load Dataframe

In [72]:
#load data in dataframe
loan_master = pd.read_csv('loan.csv',low_memory=False)

In [73]:
#check number of rows and columns
loan_master.shape

(39717, 111)

#### User Defined Functions 

In [74]:
#function to check for nulls and only display the columns having nulls 
def check_null(dataframe, percentage=True):
    if percentage==True:
        return round(dataframe.isnull().mean()[dataframe.isnull().mean()>0].apply(lambda x: x*100),3)
    else:
        return dataframe.isnull().sum()[dataframe.isnull().sum()>0]

In [75]:
#function to drop irrelevant columns
def drop_irrelevant_columns(dataframe, column_list):
    dataframe.drop(column_list, axis=1, inplace=True, errors='ignore')

In [76]:
#function to perfom data cleaning in one go
#drops columns with percentage of nulls > percentage_high parameter
#drops rows of columns having percentage of nulls < percentage_lower parameter
def dropna_in_perc_range(dataframe, percentage_low, percentage_high):
    dropna_cols_perc(dataframe, percentage_high)
    dropna_rows_perc(dataframe, percentage_low)

In [77]:
#function to drop columns based on NULLs percentage
def dropna_cols_perc(dataframe, percentage):
    n_thresh = int(len(loan_master)*(1-(percentage/100)))
    dataframe.dropna(axis=1, thresh=n_thresh, inplace=True)

In [78]:
#function to drop rows based on NULLs percentage
def dropna_rows_perc(dataframe, percentage):
    col_data = check_null(dataframe).reset_index()
    col_list = col_data['index'][col_data.iloc[:,-1]<percentage].tolist()
    dataframe.dropna(subset=col_list, inplace=True)

## Data Cleaning

In [79]:
#check columns having nulls
check_null(loan_master)

emp_title                       6.191
emp_length                      2.707
desc                           32.581
title                           0.028
mths_since_last_delinq         64.662
                               ...   
tax_liens                       0.098
tot_hi_cred_lim               100.000
total_bal_ex_mort             100.000
total_bc_limit                100.000
total_il_high_credit_limit    100.000
Length: 68, dtype: float64

##### Drop columns having ALL nulls

In [80]:
loan_master.dropna(axis=1, how='all', inplace=True)
#check shape again
loan_master.shape
#54 such columns removed

(39717, 57)

In [81]:
check_null(loan_master)

emp_title                      6.191
emp_length                     2.707
desc                          32.581
title                          0.028
mths_since_last_delinq        64.662
mths_since_last_record        92.985
revol_util                     0.126
last_pymnt_d                   0.179
next_pymnt_d                  97.130
last_credit_pull_d             0.005
collections_12_mths_ex_med     0.141
chargeoff_within_12_mths       0.141
pub_rec_bankruptcies           1.755
tax_liens                      0.098
dtype: float64

Based on percentages from null check above
##### Drop columns with >90% null rows
Columns removed: next_pymnt_d and mths_since_last_record
##### Drop rows of columns with <7% of nulls
Rows removed for columns: 
emp_title, emp_length, title, revol_util, last_pymnt_d, last_credit_pull_d,
collections_12_mths_ex_med, chargeoff_within_12_mths, pub_rec_bankruptcies, tax_liens

In [82]:
dropna_in_perc_range(loan_master, 7, 90)
#check shape again
loan_master.shape
#2 columns and some rows removed

(36431, 55)

In [83]:
check_null(loan_master)

desc                      33.008
mths_since_last_delinq    65.697
dtype: float64

##### Drop irrelevant columns

In [84]:
#list columns that are determined to be irrelevant to our analysis
list_irrelevant_cols=[]
list_irrelevant_cols.append('desc')#text entered by borrower 
list_irrelevant_cols.append('zip_code')#first 3 chars only followed by xx
list_irrelevant_cols.append('url')#loan id already available
list_irrelevant_cols.append('acc_now_delinq')#only having 0s/Nas
list_irrelevant_cols.append('title')#use purpose instead
list_irrelevant_cols.append('application_type')#all loans are individual
list_irrelevant_cols.append('chargeoff_within_12_mths')#only having 0s/Nas
list_irrelevant_cols.append('collections_12_mths_ex_med')#only having 0s/Nas
list_irrelevant_cols.append('delinq_amnt')#only having 0s/Nas
list_irrelevant_cols.append('initial_list_status')#only having value f
list_irrelevant_cols.append('policy_code')#only having value 1
list_irrelevant_cols.append('tax_liens')#only having 0s/Nas

In [86]:
drop_irrelevant_columns(loan_master,list_irrelevant_cols)

In [87]:
loan_master.shape
#43 columns left

(36431, 43)

##### Handle Data Types

In [96]:
loan_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36431 entries, 1 to 39623
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       36431 non-null  int64         
 1   member_id                36431 non-null  int64         
 2   loan_amnt                36431 non-null  int64         
 3   funded_amnt              36431 non-null  int64         
 4   funded_amnt_inv          36431 non-null  float64       
 5   term_months              36431 non-null  int64         
 6   int_rate                 36431 non-null  float64       
 7   installment              36431 non-null  float64       
 8   grade                    36431 non-null  object        
 9   sub_grade                36431 non-null  object        
 10  emp_title                36431 non-null  object        
 11  emp_length               36431 non-null  object        
 12  home_ownership           36431 n

In [88]:
#fix data type for 'term'
loan_master['term'] = loan_master['term'].apply(lambda x: str(x).replace('months','').strip()).astype('int64')
loan_master = loan_master.rename(columns={'term':'term_months'})

In [89]:
#fix data type for 'int_rate'
loan_master['int_rate'] = loan_master['int_rate'].apply(lambda x: str(x).replace('%','').strip()).astype('float64')

In [95]:
#fix data type for 'earliest_cr_line'
loan_master['earliest_cr_line'] = pd.to_datetime(loan_master['earliest_cr_line'], format='%b-%y')

dtype('<M8[ns]')

In [36]:
#fix data type emp_length
loan_master['emp_length'].value_counts()

10+ years    8407
2 years      4176
< 1 year     4052
3 years      3914
4 years      3297
5 years      3129
1 year       3026
6 years      2121
7 years      1697
8 years      1405
9 years      1207
Name: emp_length, dtype: int64

In [35]:
loan_master.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term_months,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
1,1077430,1314167,2500,2500,2500.0,60,15.27,59.83,C,C4,...,119.66,Sep-13,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
3,1076863,1277178,10000,10000,10000.0,36,13.49,339.31,C,C1,...,357.48,Apr-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
4,1075358,1311748,3000,3000,3000.0,60,12.69,67.79,B,B5,...,67.79,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
5,1075269,1311441,5000,5000,5000.0,36,7.90,156.46,A,A4,...,161.03,Jan-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
6,1069639,1304742,7000,7000,7000.0,60,15.96,170.08,C,C5,...,1313.76,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0


### Analysis

In [ ]:
#check uniqueness of id and member id
print('unique id: ',loan_master['id'].nunique())
print('unique member id: ',loan_master['member_id'].nunique())
#both are unique

In [ ]:
#create separate data frames based on loan status
charged_off_loans = loan_master[loan_master['loan_status']=='Charged Off']
current_loans = loan_master[loan_master['loan_status']=='Current']
fully_paid_loans = loan_master[loan_master['loan_status']=='Fully Paid']

In [ ]:
#avg open accounts
loan_custattr = pd.pivot_table(loan_master,index='loan_status',values='open_acc',aggfunc=np.median)
loan_custattr
#doesn't matter

In [ ]:
#avg total accounts
loan_custattr = pd.pivot_table(loan_master,index='loan_status',values='total_acc',aggfunc=np.median)
loan_custattr
#doesn't matter

In [ ]:
#verification status
charged_off_loans['verification_status'].value_counts()

In [ ]:
#verification status
fully_paid_loans['verification_status'].value_counts()

In [ ]:
#loan_master = loan_master.groupby(by='loan_status')
loan_master.head()

In [ ]:
loan_master['loan_status'].value_counts()

In [ ]:
loan_master['funded_amnt'].count()

In [ ]:
#charged_off_loans = loan_master[loan_master['loan_status']=='Charged Off']

In [ ]:
loan_master['loan_status'].value_counts()

#loan attributes 
funded amount inv
int rate
term
installment
grade
total pymt


#customer attributes
verification
annual income
emp length
purpose
addr state
total acc